In [1]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python flask

In [5]:
from kafka import KafkaProducer

topic ="smoke-test"
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
# Note: The data you send must be binary
producer.send(topic, b"Hello World!").get(timeout=30)
# Note: The data you send must be binary
producer.send(topic, b"Foo").get(timeout=30)
producer.send(topic, b"Bar").get(timeout=30)

RecordMetadata(topic='smoke-test', partition=0, topic_partition=TopicPartition(topic='smoke-test', partition=0), offset=5, timestamp=1637859369091, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=3, serialized_header_size=-1)

In [6]:
from IPython.display import display, clear_output
from time import sleep

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'
os.environ['SPARK_OPTS'] = "--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=warn"


import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[2]').getOrCreate()
sc = spark.sparkContext

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers","localhost:9092")
    .option("subscribe", "smoke-test")
    .option("startingOffsets", "earliest")
    .load()
)

smoke_test_query = (
    df.writeStream                  # Create an output stream for the query
    .outputMode("append")           # Add each msg to end of "streaming table"
    .format("memory")               # Write output stream to Spark table
    .queryName("test_query") 
    .start()
)

# Wait until query runs for a bit
sleep(10)

# Display status of query
display(smoke_test_query.status)

# Show result in Jupyter Notebook
display(
    spark.table("test_query").toPandas()
)

# Stop query
smoke_test_query.stop()


21/11/25 16:56:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5d68dab1-bc22-47aa-abc4-13cbfc6006b3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/25 16:56:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[72, 101, 108, 108, 111, 32, 87, 111, 114, 108...",smoke-test,0,0,2021-11-25 14:58:36.122,0
1,None,"[70, 111, 111]",smoke-test,0,1,2021-11-25 14:58:36.148,0
2,None,"[66, 97, 114]",smoke-test,0,2,2021-11-25 14:58:36.150,0
3,None,"[72, 101, 108, 108, 111, 32, 87, 111, 114, 108...",smoke-test,0,3,2021-11-25 16:56:09.086,0
4,None,"[70, 111, 111]",smoke-test,0,4,2021-11-25 16:56:09.089,0
5,None,"[66, 97, 114]",smoke-test,0,5,2021-11-25 16:56:09.091,0
